In [1]:
# !pip install nervaluate

In [2]:
from tqdm import tqdm
from nervaluate import Evaluator

In [3]:
with open("experiments/deberta_v3_base/2023-03-21_15-36-47/infer_text_dev.txt", "r") as f:
    test_data = f.read()

In [4]:
labels, preds = test_data.split("\n")[:2]

In [5]:
labels = labels.split("\t")[1]
preds = preds.split("\t")[1]

In [6]:
labels = labels.split(" ")
preds = preds.split(" ")

In [7]:
print(len(labels))
print(len(preds))

for i in range(10):
    print(labels[i], preds[i])

51362
51362
O O
O O
ORG ORG
O O
O O
O O
O O
O O
O O
O O


In [21]:
k = 0
for i in range(len(preds)):
    if labels[i] != preds[i]:
        k +=1
print(f" Num Errors: {k}")

 Num Errors: 279


In [8]:
def read_data(file):
    lines = open(file, "r").readlines()
    data = {"sentences": [], "labels_per_sent": []}
    sentence, labels = [], []

    for line in tqdm(lines):
        line = line.strip()

        if not line:
            if sentence and labels:
                assert len(sentence) == len(labels)
                data["sentences"].append(sentence)
                data["labels_per_sent"].append(labels)
                sentence, labels = [], []
            continue

        if line.startswith("-DOCSTART-"):
            continue
        else:
            values = line.split(" ")
            try:
                token, _, _, label = values
                sentence.append(token)
                if label != "O":
                    labels.append(label.split("-")[-1])
                else:
                    labels.append(label)

            except Exception as e:
                print(f"Error has occur: {e}")
                continue
    return data

In [9]:
orig_labels = read_data("data/eng.testa")["labels_per_sent"]

100%|██████████| 219554/219554 [00:00<00:00, 676159.30it/s]


In [22]:
true_labels = []
pred_labels = []

def conv(ls, line, k):
    return [f"I-{l}" if l != "O" else l for l in ls[k:k+len(line)]]

k = 0
for line in orig_labels:
    true_labels.append([f"I-{l}" if l != "O" else l for l in labels[k:k+len(line)]])
    pred_labels.append([f"I-{l}" if l != "O" else l for l in preds[k:k+len(line)]])
    k += len(line)

In [23]:
print(len(true_labels))
print(len(pred_labels))

for i in range(10):
    print(true_labels[i], pred_labels[i])

14041
14041
['O', 'O', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O'] ['O', 'O', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O']
['O', 'O'] ['O', 'O']
['I-LOC', 'O'] ['I-LOC', 'O']
['I-MISC', 'I-MISC', 'O', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-ORG', 'O', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'] ['I-MISC', 'I-MISC', 'O', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-ORG', 'O', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-ORG', 'O', 'I-ORG', 'O', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'I-ORG', 'O'] ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-ORG', 'O', 'I-ORG', 'O', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'I-ORG', 'O']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-ORG', 'O', 'O', 'O', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-LOC', 'I-LOC', '

In [26]:
k = 0
for i in range(len(pred_labels)):
    if true_labels[i] != pred_labels[i]:
        k +=1
print(f" Num error lines: {k}")

 Num error lines: 198


In [27]:
evaluator = Evaluator(true_labels, pred_labels, tags=["LOC", "MISC", "ORG", "PER"], loader="list")

results, results_by_tag = evaluator.evaluate()

In [28]:
results

{'ent_type': {'correct': 5943,
  'incorrect': 117,
  'partial': 0,
  'missed': 25,
  'spurious': 63,
  'possible': 6085,
  'actual': 6123,
  'precision': 0.9706026457618814,
  'recall': 0.9766639276910436,
  'f1': 0.9736238532110091},
 'partial': {'correct': 5998,
  'incorrect': 0,
  'partial': 62,
  'missed': 25,
  'spurious': 63,
  'possible': 6085,
  'actual': 6123,
  'precision': 0.9846480483423159,
  'recall': 0.990797041906327,
  'f1': 0.9877129750982963},
 'strict': {'correct': 5898,
  'incorrect': 162,
  'partial': 0,
  'missed': 25,
  'spurious': 63,
  'possible': 6085,
  'actual': 6123,
  'precision': 0.9632533072023518,
  'recall': 0.9692686935086278,
  'f1': 0.966251638269987},
 'exact': {'correct': 5998,
  'incorrect': 62,
  'partial': 0,
  'missed': 25,
  'spurious': 63,
  'possible': 6085,
  'actual': 6123,
  'precision': 0.9795851706679732,
  'recall': 0.9857025472473295,
  'f1': 0.9826343381389253}}

In [29]:
results_by_tag

{'LOC': {'ent_type': {'correct': 1812,
   'incorrect': 32,
   'partial': 0,
   'missed': 1,
   'spurious': 6,
   'possible': 1845,
   'actual': 1850,
   'precision': 0.9794594594594594,
   'recall': 0.9821138211382113,
   'f1': 0.980784844384303},
  'partial': {'correct': 1837,
   'incorrect': 0,
   'partial': 7,
   'missed': 1,
   'spurious': 6,
   'possible': 1845,
   'actual': 1850,
   'precision': 0.9948648648648649,
   'recall': 0.9975609756097561,
   'f1': 0.9962110960757782},
  'strict': {'correct': 1809,
   'incorrect': 35,
   'partial': 0,
   'missed': 1,
   'spurious': 6,
   'possible': 1845,
   'actual': 1850,
   'precision': 0.9778378378378378,
   'recall': 0.9804878048780488,
   'f1': 0.9791610284167794},
  'exact': {'correct': 1837,
   'incorrect': 7,
   'partial': 0,
   'missed': 1,
   'spurious': 6,
   'possible': 1845,
   'actual': 1850,
   'precision': 0.9929729729729729,
   'recall': 0.9956639566395664,
   'f1': 0.994316644113667}},
 'MISC': {'ent_type': {'correct': 

Источник, если интересно: https://github.com/MantisAI/nervaluate/blob/main/README.md

How can we incorporate these described scenarios into evaluation metrics? See the [original blog](http://www.davidsbatista.net/blog/2018/05/09/Named_Entity_Evaluation/) for a great explanation, a summary is included here:

We can use the following five metrics to consider difference categories of errors:

|Error type|Explanation|
|---|---|
|Correct (COR)|both are the same|
|Incorrect (INC)|the output of a system and the golden annotation don’t match|
|Partial (PAR)|system and the golden annotation are somewhat “similar” but not the same|
|Missing (MIS)|a golden annotation is not captured by a system|
|Spurius (SPU)|system produces a response which doesn’t exit in the golden annotation|

These five metrics can be measured in four different ways:

|Evaluation schema|Explanation|
|---|---|
|Strict|exact boundary surface string match and entity type|
|Exact|exact boundary match over the surface string, regardless of the type|
|Partial|partial boundary match over the surface string, regardless of the type|
|Type|some overlap between the system tagged entity and the gold annotation is required|


- Основная проблема с типом тега
- И с совмещением типа тега и его границ
- Но в целом качество хорошее